# Implement sample data for Knowledge Base DB

In [ ]:
USE master
GO

DECLARE @errorMSG varchar(300) = ''

IF DB_ID('KnowledgeBaseDB') IS NULL
   SET @errorMSG = @errorMSG + 'KnowledgeBaseDB missing' + CHAR(13)
IF DB_ID('JKU-Student-System') IS NULL
   SET @errorMSG = @errorMSG + 'JKU-Student-System missing' + CHAR(13)
IF DB_ID('MedUniWien-Student-System') IS NULL
   SET @errorMSG = @errorMSG +  'MedUniWien-Student-System missing' + CHAR(13)
IF DB_ID('TUGraz-Student-System') IS NULL
   SET @errorMSG = @errorMSG +  'TUGraz-Student-System missing' + CHAR(13)
IF DB_ID('TemplateDB') IS NULL
   SET @errorMSG = @errorMSG +  'TemplateDB missing' + CHAR(13)

IF (@errorMSG is null or @errorMSG = '')
    print 'All databases seem good to go'
else 
    print @errorMSG

## Empty and reset Knowledge Base

In [ ]:
USE KnowledgeBaseDB

DELETE FROM stage.StageJoin;

DELETE FROM kb.FillOrder;
DELETE FROM kb.ColumnConditionMap;
DELETE FROM kb.ColumnCondition;
DELETE FROM kb.JoinColumn;
DELETE FROM kb.JoinMap;
DELETE FROM kb.JoinTable;
DELETE FROM kb.TransformationFromMap;
DELETE FROM kb.TransformationOperation;
DELETE FROM kb.TableSelectMap;
DELETE FROM kb.TableSelectElement;
DELETE FROM kb.ColumnInformation;
DELETE FROM kb.TableInformation;
DELETE FROM kb.TableSelect;
DELETE FROM kb.DatabaseInformation;

DBCC CHECKIDENT ('[stage].[StageJoin]', RESEED, -1);

DBCC CHECKIDENT ('[kb].[FillOrder]', RESEED, -1);
DBCC CHECKIDENT ('[kb].[ColumnCondition]', RESEED, -1);
DBCC CHECKIDENT ('[kb].[JoinColumn]', RESEED, -1);
DBCC CHECKIDENT ('[kb].[JoinMap]', RESEED, -1);
DBCC CHECKIDENT ('[kb].[JoinTable]', RESEED, -1);
DBCC CHECKIDENT ('[kb].[TransformationOperation]', RESEED, -1);
DBCC CHECKIDENT ('[kb].[TableSelectElement]', RESEED, -1);
DBCC CHECKIDENT ('[kb].[TableSelect]', RESEED, -1);
DBCC CHECKIDENT ('[kb].[ColumnInformation]', RESEED, -1);
DBCC CHECKIDENT ('[kb].[TableInformation]', RESEED, -1);
DBCC CHECKIDENT ('[kb].[DatabaseInformation]', RESEED, -1);


## Manually load data into Knowledge Base

### Load TemplateDB information
Information about the loading target to map elements from the sources into it.



In [ ]:
USE KnowledgeBaseDB;

DECLARE @dbName VARCHAR(30) = 'TemplateDB';
DECLARE @dbID INT;

-- DB information
INSERT INTO kb.DatabaseInformation
(db_name)
VALUES (@dbName);
SET @dbID = SCOPE_IDENTITY();

-- Table information
INSERT INTO kb.TableInformation
(database_id, [schema], table_name)
SELECT @dbID, s.name, t.name
FROM [TemplateDB].sys.tables t
JOIN [TemplateDB].sys.schemas s ON s.schema_id = t.schema_id;

-- Column information
INSERT INTO kb.ColumnInformation
(table_id, column_name)
SELECT ti.table_id, c.COLUMN_NAME
FROM [TemplateDB].information_schema.COLUMNS c 
JOIN kb.DatabaseInformation di ON di.db_name = c.TABLE_CATALOG
JOIN kb.TableInformation ti ON 
    c.TABLE_NAME = ti.table_name 
    AND c.TABLE_SCHEMA = ti.[schema]
    AND ti.database_id = di.database_id

-- Get all join information
INSERT INTO stage.StageJoin
(join_type, table_1, table_2, table_condition, map_element_name, map_condition, column_1_id, column_1_element_name, column_2_id, column_2_element_name)
SELECT
    'INNER',
    ti1.table_id AS [table1 ID],
    ti2.table_id AS [table2 ID],
    '$map1',
    '$map1',
    '$col1=$col2',
    ci1.column_id AS [c1 ID],
    '$col1',
    ci2.column_id AS [c2 ID],
    '$col2'
FROM [TemplateDB].sys.foreign_key_columns fkc

INNER JOIN [TemplateDB].sys.tables tab1
    ON tab1.object_id = fkc.parent_object_id
INNER JOIN [TemplateDB].sys.tables tab2
    ON tab2.object_id = fkc.referenced_object_id

INNER JOIN [TemplateDB].sys.schemas sch1
    ON tab1.schema_id = sch1.schema_id
INNER JOIN [TemplateDB].sys.schemas sch2
    ON tab2.schema_id = sch2.schema_id

INNER JOIN [TemplateDB].sys.columns col1
    ON col1.column_id = parent_column_id AND col1.object_id = tab1.object_id
INNER JOIN [TemplateDB].sys.columns col2
    ON col2.column_id = referenced_column_id AND col2.object_id = tab2.object_id

JOIN [KnowledgeBaseDB].[kb].TableInformation ti1
    ON ti1.[schema] = sch1.[name] AND ti1.[table_name] = tab1.[name]
JOIN [KnowledgeBaseDB].[kb].TableInformation ti2
    ON ti2.[schema] = sch2.[name] AND ti2.[table_name] = tab2.[name]
JOIN [KnowledgeBaseDB].[kb].DatabaseInformation db
    ON ti1.[database_id] = db.[database_id] AND ti2.[database_id] = db.[database_id]

JOIN [KnowledgeBaseDB].[kb].ColumnInformation ci1
    ON ci1.column_name = col1.name AND ci1.table_id = ti1.table_id
JOIN [KnowledgeBaseDB].[kb].ColumnInformation ci2
    ON ci2.column_name = col2.name AND ci2.table_id = ti2.table_id

WHERE db.db_name = @dbName

### Load JKU information

In [ ]:
USE KnowledgeBaseDB;

DECLARE @dbName VARCHAR(30) = 'JKU-Student-System';
DECLARE @dbID INT;

-- DB information
INSERT INTO kb.DatabaseInformation
(db_name)
VALUES (@dbName);
SET @dbID = SCOPE_IDENTITY();

-- Table information
INSERT INTO kb.TableInformation
(database_id, [schema], table_name)
SELECT @dbID, s.name, t.name
FROM [JKU-Student-System].sys.tables t
JOIN [JKU-Student-System].sys.schemas s ON s.schema_id = t.schema_id;

-- Column information
INSERT INTO kb.ColumnInformation
(table_id, column_name)
SELECT ti.table_id, c.COLUMN_NAME
FROM [JKU-Student-System].information_schema.COLUMNS c 
JOIN kb.DatabaseInformation di ON di.db_name = c.TABLE_CATALOG
JOIN kb.TableInformation ti ON 
    c.TABLE_NAME = ti.table_name 
    AND c.TABLE_SCHEMA = ti.[schema]
    AND ti.database_id = di.database_id

-- Get all join information
INSERT INTO stage.StageJoin
(join_type, table_1, table_2, table_condition, map_element_name, map_condition, column_1_id, column_1_element_name, column_2_id, column_2_element_name)
SELECT
    'INNER',
    ti1.table_id AS [table1 ID],
    ti2.table_id AS [table2 ID],
    '$map1',
    '$map1',
    '$col1=$col2',
    ci1.column_id AS [c1 ID],
    '$col1',
    ci2.column_id AS [c2 ID],
    '$col2'
FROM [JKU-Student-System].sys.foreign_key_columns fkc

INNER JOIN [JKU-Student-System].sys.tables tab1
    ON tab1.object_id = fkc.parent_object_id
INNER JOIN [JKU-Student-System].sys.tables tab2
    ON tab2.object_id = fkc.referenced_object_id

INNER JOIN [JKU-Student-System].sys.schemas sch1
    ON tab1.schema_id = sch1.schema_id
INNER JOIN [JKU-Student-System].sys.schemas sch2
    ON tab2.schema_id = sch2.schema_id

INNER JOIN [JKU-Student-System].sys.columns col1
    ON col1.column_id = parent_column_id AND col1.object_id = tab1.object_id
INNER JOIN [JKU-Student-System].sys.columns col2
    ON col2.column_id = referenced_column_id AND col2.object_id = tab2.object_id

JOIN [KnowledgeBaseDB].[kb].TableInformation ti1
    ON ti1.[schema] = sch1.[name] AND ti1.[table_name] = tab1.[name]
JOIN [KnowledgeBaseDB].[kb].TableInformation ti2
    ON ti2.[schema] = sch2.[name] AND ti2.[table_name] = tab2.[name]
JOIN [KnowledgeBaseDB].[kb].DatabaseInformation db
    ON ti1.[database_id] = db.[database_id] AND ti2.[database_id] = db.[database_id]

JOIN [KnowledgeBaseDB].[kb].ColumnInformation ci1
    ON ci1.column_name = col1.name AND ci1.table_id = ti1.table_id
JOIN [KnowledgeBaseDB].[kb].ColumnInformation ci2
    ON ci2.column_name = col2.name AND ci2.table_id = ti2.table_id

WHERE db.db_name = @dbName
    AND ti1.table_id <> ti2.table_id -- Avoid self-references in joins

-- Remove FieldOfStudy links
DECLARE @fos_id INT = (SELECT table_id FROM kb.TableInformation ti WHERE ti.database_id = @dbID AND ti.table_name = 'FieldOfStudy' AND ti.[schema] = 'dbo');
DELETE FROM stage.StageJoin WHERE table_1 = @fos_id OR table_2 = @fos_id

-- Conditions
DECLARE @col_id INT = (SELECT column_id FROM kb.ColumnInformation ci JOIN kb.TableInformation ti ON ci.table_id = ti.table_id WHERE ci.column_name = 'grade' AND ti.database_id = @dbID AND ti.table_name = 'Registration' AND ti.[schema] = 'dbo');

INSERT INTO kb.ColumnCondition
(column_id, expression)
VALUES (@col_id, '$col IS NOT NULL')

SET @col_id = (SELECT column_id FROM kb.ColumnInformation ci JOIN kb.TableInformation ti ON ci.table_id = ti.table_id WHERE ci.column_name = 'ordering' AND ti.database_id = @dbID AND ti.table_name = 'Teaching' AND ti.[schema] = 'dbo');
INSERT INTO kb.ColumnCondition
(column_id, expression)
VALUES (@col_id, '$col = 1 OR $col IS NULL')

In [ ]:
USE KnowledgeBaseDB;

-- Define source and target database
DECLARE @source INT = (SELECT database_id FROM kb.DatabaseInformation WHERE db_name = 'JKU-Student-System');
DECLARE @target INT = (SELECT database_id FROM kb.DatabaseInformation WHERE db_name = 'TemplateDB');
DECLARE @src_col INT;
DECLARE @target_col INT;

-- *************************************
-- University table
-- *************************************
SET @target_col = (SELECT column_id FROM kb.ColumnInformation ci JOIN kb.TableInformation ti ON ci.table_id = ti.table_id WHERE ci.column_name = 'name_nk' AND ti.database_id = @target AND ti.table_name = 'University' AND ti.[schema] = 'dbo');
INSERT INTO kb.TransformationOperation
(column_id, source_database, expression)
VALUES (@target_col, @source, '''JKU-Student-System''');

-- *************************************
-- Student table
-- *************************************
SET @src_col = (SELECT column_id FROM kb.ColumnInformation ci JOIN kb.TableInformation ti ON ci.table_id = ti.table_id WHERE ci.column_name = 'gender' AND ti.database_id = @source AND ti.table_name = 'Student' AND ti.[schema] = 'dbo');
SET @target_col = (SELECT column_id FROM kb.ColumnInformation ci JOIN kb.TableInformation ti ON ci.table_id = ti.table_id WHERE ci.column_name = 'gender' AND ti.database_id = @target AND ti.table_name = 'Student' AND ti.[schema] = 'dbo');
INSERT INTO kb.TransformationOperation
(column_id, source_database, expression)
VALUES (@target_col, @source, '(Case When $col1 = 0 THEN ''Female'' when $col1 = 1 THEN ''Male''Else ''Non-Binary'' END)');
INSERT INTO kb.TransformationFromMap
(transformation_op_id, column_id, element_name)
VALUES (SCOPE_IDENTITY(), @src_col, '$col1');

SET @src_col = (SELECT column_id FROM kb.ColumnInformation ci JOIN kb.TableInformation ti ON ci.table_id = ti.table_id WHERE ci.column_name = 'country' AND ti.table_name = 'StudentAddress' AND ti.[schema] = 'dbo' AND ti.database_id = @source);
SET @target_col = (SELECT column_id FROM kb.ColumnInformation ci JOIN kb.TableInformation ti ON ci.table_id = ti.table_id WHERE ci.column_name = 'country' AND ti.table_name = 'Student' AND ti.[schema] = 'dbo' AND ti.database_id = @target);
INSERT INTO kb.TransformationOperation
(column_id, source_database, expression)
VALUES (@target_col, @source, '$col1');
INSERT INTO kb.TransformationFromMap
(transformation_op_id, column_id, element_name)
VALUES (SCOPE_IDENTITY(), @src_col, '$col1');

SET @src_col = (SELECT column_id FROM kb.ColumnInformation ci JOIN kb.TableInformation ti ON ci.table_id = ti.table_id WHERE ci.column_name = 'city' AND ti.table_name = 'StudentAddress' AND ti.[schema] = 'dbo' AND ti.database_id = @source);
SET @target_col = (SELECT column_id FROM kb.ColumnInformation ci JOIN kb.TableInformation ti ON ci.table_id = ti.table_id WHERE ci.column_name = 'city' AND ti.table_name = 'Student' AND ti.[schema] = 'dbo' AND ti.database_id = @target);
INSERT INTO kb.TransformationOperation
(column_id, source_database, expression)
VALUES (@target_col, @source, '$col1');
INSERT INTO kb.TransformationFromMap
(transformation_op_id, column_id, element_name)
VALUES (SCOPE_IDENTITY(), @src_col, '$col1');

-- *************************************
-- Study table
-- *************************************

SET @src_col = (SELECT column_id FROM kb.ColumnInformation ci JOIN kb.TableInformation ti ON ci.table_id = ti.table_id WHERE ci.column_name = 'faculty' AND ti.table_name = 'StudyProgram' AND ti.[schema] = 'dbo' AND ti.database_id = @source);
SET @target_col = (SELECT column_id FROM kb.ColumnInformation ci JOIN kb.TableInformation ti ON ci.table_id = ti.table_id WHERE ci.column_name = 'faculty' AND ti.table_name = 'Study' AND ti.[schema] = 'dbo' AND ti.database_id = @target);
INSERT INTO kb.TransformationOperation
(column_id, source_database, expression)
VALUES (@target_col, @source, '$col1');
INSERT INTO kb.TransformationFromMap
(transformation_op_id, column_id, element_name)
VALUES (SCOPE_IDENTITY(), @src_col, '$col1');

SET @src_col = (SELECT column_id FROM kb.ColumnInformation ci JOIN kb.TableInformation ti ON ci.table_id = ti.table_id WHERE ci.column_name = 'name' AND ti.table_name = 'StudyProgram' AND ti.[schema] = 'dbo' AND ti.database_id = @source);
SET @target_col = (SELECT column_id FROM kb.ColumnInformation ci JOIN kb.TableInformation ti ON ci.table_id = ti.table_id WHERE ci.column_name = 'study_program' AND ti.table_name = 'Study' AND ti.[schema] = 'dbo' AND ti.database_id = @target);
INSERT INTO kb.TransformationOperation
(column_id, source_database, expression)
VALUES (@target_col, @source, '$col1');
INSERT INTO kb.TransformationFromMap
(transformation_op_id, column_id, element_name)
VALUES (SCOPE_IDENTITY(), @src_col, '$col1');

-- *************************************
-- Exam table
-- *************************************

SET @src_col = (SELECT column_id FROM kb.ColumnInformation ci JOIN kb.TableInformation ti ON ci.table_id = ti.table_id WHERE ci.column_name = 'grade' AND ti.table_name = 'Registration' AND ti.[schema] = 'dbo' AND ti.database_id = @source);
SET @target_col = (SELECT column_id FROM kb.ColumnInformation ci JOIN kb.TableInformation ti ON ci.table_id = ti.table_id WHERE ci.column_name = 'grade' AND ti.table_name = 'Exam' AND ti.[schema] = 'dbo' AND ti.database_id = @target);
INSERT INTO kb.TransformationOperation
(column_id, source_database, expression)
VALUES (@target_col, @source, 'CAST($col1 AS INT)');
INSERT INTO kb.TransformationFromMap
(transformation_op_id, column_id, element_name)
VALUES (SCOPE_IDENTITY(), @src_col, '$col1');

SET @src_col = (SELECT column_id FROM kb.ColumnInformation ci JOIN kb.TableInformation ti ON ci.table_id = ti.table_id WHERE ci.column_name = 'Semester' AND ti.table_name = 'Registration' AND ti.[schema] = 'dbo' AND ti.database_id = @source);
SET @target_col = (SELECT column_id FROM kb.ColumnInformation ci JOIN kb.TableInformation ti ON ci.table_id = ti.table_id WHERE ci.column_name = 'year' AND ti.table_name = 'Exam' AND ti.[schema] = 'dbo' AND ti.database_id = @target);
INSERT INTO kb.TransformationOperation
(column_id, source_database, expression)
VALUES (@target_col, @source, 'RIGHT($col1, 4)');
INSERT INTO kb.TransformationFromMap
(transformation_op_id, column_id, element_name)
VALUES (SCOPE_IDENTITY(), @src_col, '$col1');

SET @src_col = (SELECT column_id FROM kb.ColumnInformation ci JOIN kb.TableInformation ti ON ci.table_id = ti.table_id WHERE ci.column_name = 'Semester' AND ti.table_name = 'Registration' AND ti.[schema] = 'dbo' AND ti.database_id = @source);
SET @target_col = (SELECT column_id FROM kb.ColumnInformation ci JOIN kb.TableInformation ti ON ci.table_id = ti.table_id WHERE ci.column_name = 'semester' AND ti.table_name = 'Exam' AND ti.[schema] = 'dbo' AND ti.database_id = @target);
INSERT INTO kb.TransformationOperation
(column_id, source_database, expression)
VALUES (@target_col, @source, '$col1');
INSERT INTO kb.TransformationFromMap
(transformation_op_id, column_id, element_name)
VALUES (SCOPE_IDENTITY(), @src_col, '$col1');

SET @target_col = (SELECT column_id FROM kb.ColumnInformation ci JOIN kb.TableInformation ti ON ci.table_id = ti.table_id WHERE ci.column_name = 'month' AND ti.table_name = 'Exam' AND ti.[schema] = 'dbo' AND ti.database_id = @target);
INSERT INTO kb.TransformationOperation
(column_id, source_database, expression)
VALUES (@target_col, @source, '''''');

-- *************************************
-- Course table
-- *************************************

SET @src_col = (SELECT column_id FROM kb.ColumnInformation ci JOIN kb.TableInformation ti ON ci.table_id = ti.table_id WHERE ci.column_name = 'Institute' AND ti.table_name = 'Professor' AND ti.[schema] = 'dbo' AND ti.database_id = @source);
SET @target_col = (SELECT column_id FROM kb.ColumnInformation ci JOIN kb.TableInformation ti ON ci.table_id = ti.table_id WHERE ci.column_name = 'Institute' AND ti.table_name = 'Course' AND ti.[schema] = 'dbo' AND ti.database_id = @target);
INSERT INTO kb.TransformationOperation
(column_id, source_database, expression)
VALUES (@target_col, @source, '$col1');
INSERT INTO kb.TransformationFromMap
(transformation_op_id, column_id, element_name)
VALUES (SCOPE_IDENTITY(), @src_col, '$col1');

SET @src_col = (SELECT column_id FROM kb.ColumnInformation ci JOIN kb.TableInformation ti ON ci.table_id = ti.table_id WHERE ci.column_name = 'course_name' AND ti.table_name = 'Course' AND ti.[schema] = 'dbo' AND ti.database_id = @source);
SET @target_col = (SELECT column_id FROM kb.ColumnInformation ci JOIN kb.TableInformation ti ON ci.table_id = ti.table_id WHERE ci.column_name = 'course_name' AND ti.table_name = 'Course' AND ti.[schema] = 'dbo' AND ti.database_id = @target);
INSERT INTO kb.TransformationOperation
(column_id, source_database, expression)
VALUES (@target_col, @source, '$col1');
INSERT INTO kb.TransformationFromMap
(transformation_op_id, column_id, element_name)
VALUES (SCOPE_IDENTITY(), @src_col, '$col1');

SET @src_col = (SELECT column_id FROM kb.ColumnInformation ci JOIN kb.TableInformation ti ON ci.table_id = ti.table_id WHERE ci.column_name = 'ects' AND ti.table_name = 'Course' AND ti.[schema] = 'dbo' AND ti.database_id = @source);
SET @target_col = (SELECT column_id FROM kb.ColumnInformation ci JOIN kb.TableInformation ti ON ci.table_id = ti.table_id WHERE ci.column_name = 'ects' AND ti.table_name = 'Course' AND ti.[schema] = 'dbo' AND ti.database_id = @target);
INSERT INTO kb.TransformationOperation
(column_id, source_database, expression)
VALUES (@target_col, @source, '$col1');
INSERT INTO kb.TransformationFromMap
(transformation_op_id, column_id, element_name)
VALUES (SCOPE_IDENTITY(), @src_col, '$col1');

### Load TUGraz information

In [ ]:
USE KnowledgeBaseDB;

DECLARE @dbName VARCHAR(30) = 'TUGraz-Student-System';
DECLARE @dbID INT;

-- DB information
INSERT INTO kb.DatabaseInformation
(db_name)
VALUES (@dbName);
SET @dbID = SCOPE_IDENTITY();

-- Table information
INSERT INTO kb.TableInformation
(database_id, [schema], table_name)
SELECT @dbID, s.name, t.name
FROM [TUGraz-Student-System].sys.tables t
JOIN [TUGraz-Student-System].sys.schemas s ON s.schema_id = t.schema_id;

-- Column information
INSERT INTO kb.ColumnInformation
(table_id, column_name)
SELECT ti.table_id, c.COLUMN_NAME
FROM [TUGraz-Student-System].information_schema.COLUMNS c 
JOIN kb.DatabaseInformation di ON di.db_name = c.TABLE_CATALOG
JOIN kb.TableInformation ti ON 
    c.TABLE_NAME = ti.table_name 
    AND c.TABLE_SCHEMA = ti.[schema]
    AND ti.database_id = di.database_id

-- Get all join information
INSERT INTO stage.StageJoin
(join_type, table_1, table_2, table_condition, map_element_name, map_condition, column_1_id, column_1_element_name, column_2_id, column_2_element_name)
SELECT 
    'INNER',
    ti1.table_id AS [table1 ID],
    ti2.table_id AS [table2 ID],
    '$map1',
    '$map1',
    '$col1=$col2',
    ci1.column_id AS [c1 ID],
    '$col1',
    ci2.column_id AS [c2 ID],
    '$col2'
FROM [TUGraz-Student-System].sys.foreign_key_columns fkc

INNER JOIN [TUGraz-Student-System].sys.tables tab1
    ON tab1.object_id = fkc.parent_object_id
INNER JOIN [TUGraz-Student-System].sys.tables tab2
    ON tab2.object_id = fkc.referenced_object_id

INNER JOIN [TUGraz-Student-System].sys.schemas sch1
    ON tab1.schema_id = sch1.schema_id
INNER JOIN [TUGraz-Student-System].sys.schemas sch2
    ON tab2.schema_id = sch2.schema_id

INNER JOIN [TUGraz-Student-System].sys.columns col1
    ON col1.column_id = parent_column_id AND col1.object_id = tab1.object_id
INNER JOIN [TUGraz-Student-System].sys.columns col2
    ON col2.column_id = referenced_column_id AND col2.object_id = tab2.object_id

JOIN [KnowledgeBaseDB].[kb].TableInformation ti1
    ON ti1.[schema] = sch1.[name] AND ti1.[table_name] = tab1.[name]
JOIN [KnowledgeBaseDB].[kb].TableInformation ti2
    ON ti2.[schema] = sch2.[name] AND ti2.[table_name] = tab2.[name]
JOIN [KnowledgeBaseDB].[kb].DatabaseInformation db
    ON ti1.[database_id] = db.[database_id] AND ti2.[database_id] = db.[database_id]

JOIN [KnowledgeBaseDB].[kb].ColumnInformation ci1
    ON ci1.column_name = col1.name AND ci1.table_id = ti1.table_id
JOIN [KnowledgeBaseDB].[kb].ColumnInformation ci2
    ON ci2.column_name = col2.name AND ci2.table_id = ti2.table_id

WHERE db.db_name = @dbName
    AND ti1.table_id <> ti2.table_id -- Avoid self-references in joins

-- Conditions
DECLARE @col_id INT = (SELECT column_id FROM kb.ColumnInformation ci JOIN kb.TableInformation ti ON ci.table_id = ti.table_id WHERE ci.column_name = 'grade' AND ti.database_id = @dbID AND ti.table_name = 'ExamRegistration' AND ti.[schema] = 'dbo');

INSERT INTO kb.ColumnCondition
(column_id, expression)
VALUES (@col_id, '$col IS NOT NULL')

-- Subselect
DECLARE @as_col INT = (SELECT column_id FROM kb.ColumnInformation ci JOIN kb.TableInformation ti ON ci.table_id = ti.table_id WHERE ci.column_name = 'time_from' AND ti.database_id = @dbID AND ti.table_name = 'Exam' AND ti.[schema] = 'dbo');
DECLARE @tbl_id INT = (SELECT table_id FROM kb.TableInformation ti WHERE ti.database_id = @dbID AND ti.table_name = 'Exam' AND ti.[schema] = 'dbo');

DECLARE @col2 INT = (SELECT column_id FROM kb.ColumnInformation ci JOIN kb.TableInformation ti ON ci.table_id = ti.table_id WHERE ci.column_name = 'exam_id' AND ti.database_id = @dbID AND ti.table_name = 'Exam' AND ti.[schema] = 'dbo');
DECLARE @col3 INT = (SELECT column_id FROM kb.ColumnInformation ci JOIN kb.TableInformation ti ON ci.table_id = ti.table_id WHERE ci.column_name = 'course_id' AND ti.database_id = @dbID AND ti.table_name = 'Exam' AND ti.[schema] = 'dbo');


INSERT INTO kb.TableSelect
(comment)
VALUES ('Select to convert a unix timestamp to datetime');
DECLARE @ts_id INT = SCOPE_IDENTITY();

INSERT INTO kb.TableSelectElement
(table_select_id, expression, as_name)
VALUES (@ts_id, 'dateadd(S, [$col1], ''1970-01-01'') AS ''$as_col'', [$col2], [$col3]', @as_col)
DECLARE @tse_id INT = SCOPE_IDENTITY();

INSERT INTO kb.TableSelectMap
(column_id, element_name, table_select_element_id)
VALUES (@as_col, '$col1', @tse_id),
(@col2, '$col2', @tse_id),
(@col3, '$col3', @tse_id)

-- Update table with subselect id
UPDATE ti
SET ti.table_select_id = @ts_id
FROM kb.TableInformation ti
    WHERE ti.table_id = @tbl_id

In [ ]:
USE KnowledgeBaseDB;

-- Define source and target database
DECLARE @source INT = (SELECT database_id FROM kb.DatabaseInformation WHERE db_name = 'TUGraz-Student-System');
DECLARE @target INT = (SELECT database_id FROM kb.DatabaseInformation WHERE db_name = 'TemplateDB');
DECLARE @src_col INT;
DECLARE @target_col INT;

-- *************************************
-- University table
-- *************************************
SET @target_col = (SELECT column_id FROM kb.ColumnInformation ci JOIN kb.TableInformation ti ON ci.table_id = ti.table_id WHERE ci.column_name = 'name_nk' AND ti.database_id = @target AND ti.table_name = 'University' AND ti.[schema] = 'dbo');
INSERT INTO kb.TransformationOperation
(column_id, source_database, expression)
VALUES (@target_col, @source, '''TUGraz-Student-System''');

-- *************************************
-- Student table
-- *************************************
SET @src_col = (SELECT column_id FROM kb.ColumnInformation ci JOIN kb.TableInformation ti ON ci.table_id = ti.table_id WHERE ci.column_name = 'gender' AND ti.database_id = @source AND ti.table_name = 'Student' AND ti.[schema] = 'dbo');
SET @target_col = (SELECT column_id FROM kb.ColumnInformation ci JOIN kb.TableInformation ti ON ci.table_id = ti.table_id WHERE ci.column_name = 'gender' AND ti.database_id = @target AND ti.table_name = 'Student' AND ti.[schema] = 'dbo');
INSERT INTO kb.TransformationOperation
(column_id, source_database, expression)
VALUES (@target_col, @source, '(Case When $col1 = 1 THEN ''Female'' when $col1 = 0 THEN ''Male''Else ''Non-Binary'' END)');
INSERT INTO kb.TransformationFromMap
(transformation_op_id, column_id, element_name)
VALUES (SCOPE_IDENTITY(), @src_col, '$col1');

SET @src_col = (SELECT column_id FROM kb.ColumnInformation ci JOIN kb.TableInformation ti ON ci.table_id = ti.table_id WHERE ci.column_name = 'full_address' AND ti.table_name = 'Student' AND ti.[schema] = 'dbo' AND ti.database_id = @source);
SET @target_col = (SELECT column_id FROM kb.ColumnInformation ci JOIN kb.TableInformation ti ON ci.table_id = ti.table_id WHERE ci.column_name = 'country' AND ti.table_name = 'Student' AND ti.[schema] = 'dbo' AND ti.database_id = @target);
INSERT INTO kb.TransformationOperation
(column_id, source_database, expression)
VALUES (@target_col, @source, '(SELECT nicename FROM [TemplateDB].[stage].[Stage_CountryCodeMap] WHERE iso = (RIGHT($col1, LEN($col1) -1 - charindex('','', $col1, charindex('','', $col1, charindex('','', $col1, 1) +1) +1))))');
INSERT INTO kb.TransformationFromMap
(transformation_op_id, column_id, element_name)
VALUES (SCOPE_IDENTITY(), @src_col, '$col1');

SET @src_col = (SELECT column_id FROM kb.ColumnInformation ci JOIN kb.TableInformation ti ON ci.table_id = ti.table_id WHERE ci.column_name = 'full_address' AND ti.table_name = 'Student' AND ti.[schema] = 'dbo' AND ti.database_id = @source);
SET @target_col = (SELECT column_id FROM kb.ColumnInformation ci JOIN kb.TableInformation ti ON ci.table_id = ti.table_id WHERE ci.column_name = 'city' AND ti.table_name = 'Student' AND ti.[schema] = 'dbo' AND ti.database_id = @target);
INSERT INTO kb.TransformationOperation
(column_id, source_database, expression)
VALUES (@target_col, @source, 'SUBSTRING($col1, charindex('','', $col1, charindex('','', $col1, 1)+1)+2, charindex('','', $col1, charindex('','', $col1, charindex('','', $col1, 1) +1) +1)-charindex('','', $col1, charindex('','', $col1, 1)+1)-2)');
INSERT INTO kb.TransformationFromMap
(transformation_op_id, column_id, element_name)
VALUES (SCOPE_IDENTITY(), @src_col, '$col1');

-- *************************************
-- Study table
-- *************************************

SET @src_col = (SELECT column_id FROM kb.ColumnInformation ci JOIN kb.TableInformation ti ON ci.table_id = ti.table_id WHERE ci.column_name = 'name' AND ti.table_name = 'Faculty' AND ti.[schema] = 'dbo' AND ti.database_id = @source);
SET @target_col = (SELECT column_id FROM kb.ColumnInformation ci JOIN kb.TableInformation ti ON ci.table_id = ti.table_id WHERE ci.column_name = 'faculty' AND ti.table_name = 'Study' AND ti.[schema] = 'dbo' AND ti.database_id = @target);
INSERT INTO kb.TransformationOperation
(column_id, source_database, expression)
VALUES (@target_col, @source, '$col1');
INSERT INTO kb.TransformationFromMap
(transformation_op_id, column_id, element_name)
VALUES (SCOPE_IDENTITY(), @src_col, '$col1');

SET @src_col = (SELECT column_id FROM kb.ColumnInformation ci JOIN kb.TableInformation ti ON ci.table_id = ti.table_id WHERE ci.column_name = 'name' AND ti.table_name = 'StudyField' AND ti.[schema] = 'dbo' AND ti.database_id = @source);
SET @target_col = (SELECT column_id FROM kb.ColumnInformation ci JOIN kb.TableInformation ti ON ci.table_id = ti.table_id WHERE ci.column_name = 'study_program' AND ti.table_name = 'Study' AND ti.[schema] = 'dbo' AND ti.database_id = @target);
INSERT INTO kb.TransformationOperation
(column_id, source_database, expression)
VALUES (@target_col, @source, '$col1');
INSERT INTO kb.TransformationFromMap
(transformation_op_id, column_id, element_name)
VALUES (SCOPE_IDENTITY(), @src_col, '$col1');

-- *************************************
-- Exam table
-- *************************************

SET @src_col = (SELECT column_id FROM kb.ColumnInformation ci JOIN kb.TableInformation ti ON ci.table_id = ti.table_id WHERE ci.column_name = 'grade' AND ti.table_name = 'ExamRegistration' AND ti.[schema] = 'dbo' AND ti.database_id = @source);
SET @target_col = (SELECT column_id FROM kb.ColumnInformation ci JOIN kb.TableInformation ti ON ci.table_id = ti.table_id WHERE ci.column_name = 'grade' AND ti.table_name = 'Exam' AND ti.[schema] = 'dbo' AND ti.database_id = @target);
INSERT INTO kb.TransformationOperation
(column_id, source_database, expression)
VALUES (@target_col, @source, '$col1');
INSERT INTO kb.TransformationFromMap
(transformation_op_id, column_id, element_name)
VALUES (SCOPE_IDENTITY(), @src_col, '$col1');

SET @src_col = (SELECT column_id FROM kb.ColumnInformation ci JOIN kb.TableInformation ti ON ci.table_id = ti.table_id WHERE ci.column_name = 'time_from' AND ti.table_name = 'Exam' AND ti.[schema] = 'dbo' AND ti.database_id = @source);
SET @target_col = (SELECT column_id FROM kb.ColumnInformation ci JOIN kb.TableInformation ti ON ci.table_id = ti.table_id WHERE ci.column_name = 'year' AND ti.table_name = 'Exam' AND ti.[schema] = 'dbo' AND ti.database_id = @target);
INSERT INTO kb.TransformationOperation
(column_id, source_database, expression)
VALUES (@target_col, @source, 'YEAR($col1)');
INSERT INTO kb.TransformationFromMap
(transformation_op_id, column_id, element_name)
VALUES (SCOPE_IDENTITY(), @src_col, '$col1');

SET @src_col = (SELECT column_id FROM kb.ColumnInformation ci JOIN kb.TableInformation ti ON ci.table_id = ti.table_id WHERE ci.column_name = 'time_from' AND ti.table_name = 'Exam' AND ti.[schema] = 'dbo' AND ti.database_id = @source);
SET @target_col = (SELECT column_id FROM kb.ColumnInformation ci JOIN kb.TableInformation ti ON ci.table_id = ti.table_id WHERE ci.column_name = 'semester' AND ti.table_name = 'Exam' AND ti.[schema] = 'dbo' AND ti.database_id = @target);
INSERT INTO kb.TransformationOperation
(column_id, source_database, expression)
VALUES (@target_col, @source, '(CASE WHEN MONTH($col1) <= 2 THEN ''WS'' + CONVERT(varchar, YEAR($col1 - 1)) WHEN MONTH($col1) <= 9 THEN ''SS'' + CONVERT(varchar, YEAR($col1)) ELSE ''WS'' + CONVERT(varchar, YEAR($col1)) END)');
INSERT INTO kb.TransformationFromMap
(transformation_op_id, column_id, element_name)
VALUES (SCOPE_IDENTITY(), @src_col, '$col1');

SET @src_col = (SELECT column_id FROM kb.ColumnInformation ci JOIN kb.TableInformation ti ON ci.table_id = ti.table_id WHERE ci.column_name = 'time_from' AND ti.table_name = 'Exam' AND ti.[schema] = 'dbo' AND ti.database_id = @source);
SET @target_col = (SELECT column_id FROM kb.ColumnInformation ci JOIN kb.TableInformation ti ON ci.table_id = ti.table_id WHERE ci.column_name = 'month' AND ti.table_name = 'Exam' AND ti.[schema] = 'dbo' AND ti.database_id = @target);
INSERT INTO kb.TransformationOperation
(column_id, source_database, expression)
VALUES (@target_col, @source, 'DateName( month , DateAdd( month , MONTH($col1) , -1 ))');
INSERT INTO kb.TransformationFromMap
(transformation_op_id, column_id, element_name)
VALUES (SCOPE_IDENTITY(), @src_col, '$col1');

-- *************************************
-- Course table
-- *************************************

SET @src_col = (SELECT column_id FROM kb.ColumnInformation ci JOIN kb.TableInformation ti ON ci.table_id = ti.table_id WHERE ci.column_name = 'institute' AND ti.table_name = 'Course' AND ti.[schema] = 'dbo' AND ti.database_id = @source);
SET @target_col = (SELECT column_id FROM kb.ColumnInformation ci JOIN kb.TableInformation ti ON ci.table_id = ti.table_id WHERE ci.column_name = 'Institute' AND ti.table_name = 'Course' AND ti.[schema] = 'dbo' AND ti.database_id = @target);
INSERT INTO kb.TransformationOperation
(column_id, source_database, expression)
VALUES (@target_col, @source, '$col1');
INSERT INTO kb.TransformationFromMap
(transformation_op_id, column_id, element_name)
VALUES (SCOPE_IDENTITY(), @src_col, '$col1');

SET @src_col = (SELECT column_id FROM kb.ColumnInformation ci JOIN kb.TableInformation ti ON ci.table_id = ti.table_id WHERE ci.column_name = 'name' AND ti.table_name = 'Course' AND ti.[schema] = 'dbo' AND ti.database_id = @source);
SET @target_col = (SELECT column_id FROM kb.ColumnInformation ci JOIN kb.TableInformation ti ON ci.table_id = ti.table_id WHERE ci.column_name = 'course_name' AND ti.table_name = 'Course' AND ti.[schema] = 'dbo' AND ti.database_id = @target);
INSERT INTO kb.TransformationOperation
(column_id, source_database, expression)
VALUES (@target_col, @source, '$col1');
INSERT INTO kb.TransformationFromMap
(transformation_op_id, column_id, element_name)
VALUES (SCOPE_IDENTITY(), @src_col, '$col1');

SET @src_col = (SELECT column_id FROM kb.ColumnInformation ci JOIN kb.TableInformation ti ON ci.table_id = ti.table_id WHERE ci.column_name = 'ects' AND ti.table_name = 'Course' AND ti.[schema] = 'dbo' AND ti.database_id = @source);
SET @target_col = (SELECT column_id FROM kb.ColumnInformation ci JOIN kb.TableInformation ti ON ci.table_id = ti.table_id WHERE ci.column_name = 'ects' AND ti.table_name = 'Course' AND ti.[schema] = 'dbo' AND ti.database_id = @target);
INSERT INTO kb.TransformationOperation
(column_id, source_database, expression)
VALUES (@target_col, @source, '$col1');
INSERT INTO kb.TransformationFromMap
(transformation_op_id, column_id, element_name)
VALUES (SCOPE_IDENTITY(), @src_col, '$col1');

### Load MedUniWien information

In [ ]:
USE KnowledgeBaseDB;

DECLARE @dbName VARCHAR(30) = 'MedUniWien-Student-System';
DECLARE @dbID INT;

-- DB information
INSERT INTO kb.DatabaseInformation
(db_name)
VALUES (@dbName);
SET @dbID = SCOPE_IDENTITY();

-- Table information
INSERT INTO kb.TableInformation
(database_id, [schema], table_name)
SELECT @dbID, s.name, t.name
FROM [MedUniWien-Student-System].sys.tables t
JOIN [MedUniWien-Student-System].sys.schemas s ON s.schema_id = t.schema_id;

-- Column information
INSERT INTO kb.ColumnInformation
(table_id, column_name)
SELECT ti.table_id, c.COLUMN_NAME
FROM [MedUniWien-Student-System].information_schema.COLUMNS c 
JOIN kb.DatabaseInformation di ON di.db_name = c.TABLE_CATALOG
JOIN kb.TableInformation ti ON 
    c.TABLE_NAME = ti.table_name 
    AND c.TABLE_SCHEMA = ti.[schema]
    AND ti.database_id = di.database_id

-- Get all join information
INSERT INTO stage.StageJoin
(join_type, table_1, table_2, table_condition, map_element_name, map_condition, column_1_id, column_1_element_name, column_2_id, column_2_element_name)
SELECT 
    'INNER',
    ti1.table_id AS [table1 ID],
    ti2.table_id AS [table2 ID],
    '$map1',
    '$map1',
    '$col1=$col2',
    ci1.column_id AS [c1 ID],
    '$col1',
    ci2.column_id AS [c2 ID],
    '$col2'
FROM [MedUniWien-Student-System].sys.foreign_key_columns fkc

INNER JOIN [MedUniWien-Student-System].sys.tables tab1
    ON tab1.object_id = fkc.parent_object_id
INNER JOIN [MedUniWien-Student-System].sys.tables tab2
    ON tab2.object_id = fkc.referenced_object_id

INNER JOIN [MedUniWien-Student-System].sys.schemas sch1
    ON tab1.schema_id = sch1.schema_id
INNER JOIN [MedUniWien-Student-System].sys.schemas sch2
    ON tab2.schema_id = sch2.schema_id

INNER JOIN [MedUniWien-Student-System].sys.columns col1
    ON col1.column_id = parent_column_id AND col1.object_id = tab1.object_id
INNER JOIN [MedUniWien-Student-System].sys.columns col2
    ON col2.column_id = referenced_column_id AND col2.object_id = tab2.object_id

JOIN [KnowledgeBaseDB].[kb].TableInformation ti1
    ON ti1.[schema] = sch1.[name] AND ti1.[table_name] = tab1.[name]
JOIN [KnowledgeBaseDB].[kb].TableInformation ti2
    ON ti2.[schema] = sch2.[name] AND ti2.[table_name] = tab2.[name]
JOIN [KnowledgeBaseDB].[kb].DatabaseInformation db
    ON ti1.[database_id] = db.[database_id] AND ti2.[database_id] = db.[database_id]

JOIN [KnowledgeBaseDB].[kb].ColumnInformation ci1
    ON ci1.column_name = col1.name AND ci1.table_id = ti1.table_id
JOIN [KnowledgeBaseDB].[kb].ColumnInformation ci2
    ON ci2.column_name = col2.name AND ci2.table_id = ti2.table_id

WHERE db.db_name = @dbName
    AND ti1.table_id <> ti2.table_id -- Avoid self-references in joins

-- Conditions
DECLARE @col_id INT = (SELECT column_id FROM kb.ColumnInformation ci JOIN kb.TableInformation ti ON ci.table_id = ti.table_id WHERE ci.column_name = 'note' AND ti.database_id = @dbID AND ti.table_name = 'PruefungRegistrierung' AND ti.[schema] = 'dbo');

INSERT INTO kb.ColumnCondition
(column_id, expression)
VALUES (@col_id, '$col IS NOT NULL')


In [ ]:
USE KnowledgeBaseDB;

-- Define source and target database
DECLARE @source INT = (SELECT database_id FROM kb.DatabaseInformation WHERE db_name = 'MedUniWien-Student-System');
DECLARE @target INT = (SELECT database_id FROM kb.DatabaseInformation WHERE db_name = 'TemplateDB');
DECLARE @src_col INT;
DECLARE @target_col INT;

-- *************************************
-- University table
-- *************************************
SET @target_col = (SELECT column_id FROM kb.ColumnInformation ci JOIN kb.TableInformation ti ON ci.table_id = ti.table_id WHERE ci.column_name = 'name_nk' AND ti.database_id = @target AND ti.table_name = 'University' AND ti.[schema] = 'dbo');
INSERT INTO kb.TransformationOperation
(column_id, source_database, expression)
VALUES (@target_col, @source,  '''MedUniWien''');

-- *************************************
-- Student table
-- *************************************
SET @src_col = (SELECT column_id FROM kb.ColumnInformation ci JOIN kb.TableInformation ti ON ci.table_id = ti.table_id WHERE ci.column_name = 'geschlecht' AND ti.database_id = @source AND ti.table_name = 'Studierende' AND ti.[schema] = 'dbo');
SET @target_col = (SELECT column_id FROM kb.ColumnInformation ci JOIN kb.TableInformation ti ON ci.table_id = ti.table_id WHERE ci.column_name = 'gender' AND ti.database_id = @target AND ti.table_name = 'Student' AND ti.[schema] = 'dbo');
INSERT INTO kb.TransformationOperation
(column_id, source_database, expression)
VALUES (@target_col, @source,  '(Case When $col1 = ''Weiblich'' THEN ''Female'' when $col1 = ''Männlich'' THEN ''Male'' Else ''Non-Binary'' END )');
INSERT INTO kb.TransformationFromMap
(transformation_op_id, column_id, element_name)
VALUES (SCOPE_IDENTITY(), @src_col, '$col1');

SET @src_col = (SELECT column_id FROM kb.ColumnInformation ci JOIN kb.TableInformation ti ON ci.table_id = ti.table_id WHERE ci.column_name = 'land' AND ti.table_name = 'Studierende' AND ti.[schema] = 'dbo' AND ti.database_id = @source);
SET @target_col = (SELECT column_id FROM kb.ColumnInformation ci JOIN kb.TableInformation ti ON ci.table_id = ti.table_id WHERE ci.column_name = 'country' AND ti.table_name = 'Student' AND ti.[schema] = 'dbo' AND ti.database_id = @target);
INSERT INTO kb.TransformationOperation
(column_id, source_database, expression)
VALUES (@target_col, @source,  '(SELECT nicename FROM [TemplateDB].[stage].Stage_CountryCodeMap WHERE iso = $col1)');
INSERT INTO kb.TransformationFromMap
(transformation_op_id, column_id, element_name)
VALUES (SCOPE_IDENTITY(), @src_col, '$col1');

SET @src_col = (SELECT column_id FROM kb.ColumnInformation ci JOIN kb.TableInformation ti ON ci.table_id = ti.table_id WHERE ci.column_name = 'stadt' AND ti.table_name = 'Studierende' AND ti.[schema] = 'dbo' AND ti.database_id = @source);
SET @target_col = (SELECT column_id FROM kb.ColumnInformation ci JOIN kb.TableInformation ti ON ci.table_id = ti.table_id WHERE ci.column_name = 'city' AND ti.table_name = 'Student' AND ti.[schema] = 'dbo' AND ti.database_id = @target);
INSERT INTO kb.TransformationOperation
(column_id, source_database, expression)
VALUES (@target_col, @source,  '$col1');
INSERT INTO kb.TransformationFromMap
(transformation_op_id, column_id, element_name)
VALUES (SCOPE_IDENTITY(), @src_col, '$col1');

-- *************************************
-- Study table
-- *************************************
SET @src_col = (SELECT column_id FROM kb.ColumnInformation ci JOIN kb.TableInformation ti ON ci.table_id = ti.table_id WHERE ci.column_name = 'fakultaet' AND ti.table_name = 'Studium' AND ti.[schema] = 'dbo' AND ti.database_id = @source);
SET @target_col = (SELECT column_id FROM kb.ColumnInformation ci JOIN kb.TableInformation ti ON ci.table_id = ti.table_id WHERE ci.column_name = 'faculty' AND ti.table_name = 'Study' AND ti.[schema] = 'dbo' AND ti.database_id = @target);
INSERT INTO kb.TransformationOperation
(column_id, source_database, expression)
VALUES (@target_col, @source,  '$col1');
INSERT INTO kb.TransformationFromMap
(transformation_op_id, column_id, element_name)
VALUES (SCOPE_IDENTITY(), @src_col, '$col1');

SET @src_col = (SELECT column_id FROM kb.ColumnInformation ci JOIN kb.TableInformation ti ON ci.table_id = ti.table_id WHERE ci.column_name = 'studium_name' AND ti.table_name = 'Studium' AND ti.[schema] = 'dbo' AND ti.database_id = @source);
SET @target_col = (SELECT column_id FROM kb.ColumnInformation ci JOIN kb.TableInformation ti ON ci.table_id = ti.table_id WHERE ci.column_name = 'study_program' AND ti.table_name = 'Study' AND ti.[schema] = 'dbo' AND ti.database_id = @target);
INSERT INTO kb.TransformationOperation
(column_id, source_database, expression)
VALUES (@target_col, @source,  '$col1');
INSERT INTO kb.TransformationFromMap
(transformation_op_id, column_id, element_name)
VALUES (SCOPE_IDENTITY(), @src_col, '$col1');

-- *************************************
-- Exam table
-- *************************************

SET @src_col = (SELECT column_id FROM kb.ColumnInformation ci JOIN kb.TableInformation ti ON ci.table_id = ti.table_id WHERE ci.column_name = 'note' AND ti.table_name = 'PruefungRegistrierung' AND ti.[schema] = 'dbo' AND ti.database_id = @source);
SET @target_col = (SELECT column_id FROM kb.ColumnInformation ci JOIN kb.TableInformation ti ON ci.table_id = ti.table_id WHERE ci.column_name = 'grade' AND ti.table_name = 'Exam' AND ti.[schema] = 'dbo' AND ti.database_id = @target);
INSERT INTO kb.TransformationOperation
(column_id, source_database, expression)
VALUES (@target_col, @source,  '$col1');
INSERT INTO kb.TransformationFromMap
(transformation_op_id, column_id, element_name)
VALUES (SCOPE_IDENTITY(), @src_col, '$col1');

SET @src_col = (SELECT column_id FROM kb.ColumnInformation ci JOIN kb.TableInformation ti ON ci.table_id = ti.table_id WHERE ci.column_name = 'datum' AND ti.table_name = 'Pruefung' AND ti.[schema] = 'dbo' AND ti.database_id = @source);
SET @target_col = (SELECT column_id FROM kb.ColumnInformation ci JOIN kb.TableInformation ti ON ci.table_id = ti.table_id WHERE ci.column_name = 'year' AND ti.table_name = 'Exam' AND ti.[schema] = 'dbo' AND ti.database_id = @target);
INSERT INTO kb.TransformationOperation
(column_id, source_database, expression)
VALUES (@target_col, @source,  'CONVERT(int, LEFT($col1, 4))');
INSERT INTO kb.TransformationFromMap
(transformation_op_id, column_id, element_name)
VALUES (SCOPE_IDENTITY(), @src_col, '$col1');

SET @src_col = (SELECT column_id FROM kb.ColumnInformation ci JOIN kb.TableInformation ti ON ci.table_id = ti.table_id WHERE ci.column_name = 'datum' AND ti.table_name = 'Pruefung' AND ti.[schema] = 'dbo' AND ti.database_id = @source);
SET @target_col = (SELECT column_id FROM kb.ColumnInformation ci JOIN kb.TableInformation ti ON ci.table_id = ti.table_id WHERE ci.column_name = 'semester' AND ti.table_name = 'Exam' AND ti.[schema] = 'dbo' AND ti.database_id = @target);
INSERT INTO kb.TransformationOperation
(column_id, source_database, expression)
VALUES (@target_col, @source,  '(Case When CONVERT(int, SUBSTRING($col1, 5, 2)) <= 2 THEN ''WS'' + CONVERT(varchar, (CONVERT(int, LEFT($col1, 4)) - 1))  when CONVERT(int, SUBSTRING($col1, 5, 2)) <= 9 THEN ''SS'' + LEFT($col1, 4) Else ''WS'' + LEFT($col1, 4) END )');
INSERT INTO kb.TransformationFromMap
(transformation_op_id, column_id, element_name)
VALUES (SCOPE_IDENTITY(), @src_col, '$col1');

SET @src_col = (SELECT column_id FROM kb.ColumnInformation ci JOIN kb.TableInformation ti ON ci.table_id = ti.table_id WHERE ci.column_name = 'datum' AND ti.table_name = 'Pruefung' AND ti.[schema] = 'dbo' AND ti.database_id = @source);
SET @target_col = (SELECT column_id FROM kb.ColumnInformation ci JOIN kb.TableInformation ti ON ci.table_id = ti.table_id WHERE ci.column_name = 'month' AND ti.table_name = 'Exam' AND ti.[schema] = 'dbo' AND ti.database_id = @target);
INSERT INTO kb.TransformationOperation
(column_id, source_database, expression)
VALUES (@target_col, @source,  'DateName( month , DateAdd( month , CONVERT(int, SUBSTRING($col1, 5, 2)) , -1 ) )');
INSERT INTO kb.TransformationFromMap
(transformation_op_id, column_id, element_name)
VALUES (SCOPE_IDENTITY(), @src_col, '$col1');

-- *************************************
-- Course table
-- *************************************

SET @src_col = (SELECT column_id FROM kb.ColumnInformation ci JOIN kb.TableInformation ti ON ci.table_id = ti.table_id WHERE ci.column_name = 'institut' AND ti.table_name = 'Lehrveranstaltung' AND ti.[schema] = 'dbo' AND ti.database_id = @source);
SET @target_col = (SELECT column_id FROM kb.ColumnInformation ci JOIN kb.TableInformation ti ON ci.table_id = ti.table_id WHERE ci.column_name = 'Institute' AND ti.table_name = 'Course' AND ti.[schema] = 'dbo' AND ti.database_id = @target);
INSERT INTO kb.TransformationOperation
(column_id, source_database, expression)
VALUES (@target_col, @source,  '$col1');
INSERT INTO kb.TransformationFromMap
(transformation_op_id, column_id, element_name)
VALUES (SCOPE_IDENTITY(), @src_col, '$col1');

SET @src_col = (SELECT column_id FROM kb.ColumnInformation ci JOIN kb.TableInformation ti ON ci.table_id = ti.table_id WHERE ci.column_name = 'name' AND ti.table_name = 'Lehrveranstaltung' AND ti.[schema] = 'dbo' AND ti.database_id = @source);
SET @target_col = (SELECT column_id FROM kb.ColumnInformation ci JOIN kb.TableInformation ti ON ci.table_id = ti.table_id WHERE ci.column_name = 'course_name' AND ti.table_name = 'Course' AND ti.[schema] = 'dbo' AND ti.database_id = @target);
INSERT INTO kb.TransformationOperation
(column_id, source_database, expression)
VALUES (@target_col, @source,  '$col1');
INSERT INTO kb.TransformationFromMap
(transformation_op_id, column_id, element_name)
VALUES (SCOPE_IDENTITY(), @src_col, '$col1');

SET @src_col = (SELECT column_id FROM kb.ColumnInformation ci JOIN kb.TableInformation ti ON ci.table_id = ti.table_id WHERE ci.column_name = 'ects' AND ti.table_name = 'Lehrveranstaltung' AND ti.[schema] = 'dbo' AND ti.database_id = @source);
SET @target_col = (SELECT column_id FROM kb.ColumnInformation ci JOIN kb.TableInformation ti ON ci.table_id = ti.table_id WHERE ci.column_name = 'ects' AND ti.table_name = 'Course' AND ti.[schema] = 'dbo' AND ti.database_id = @target);
INSERT INTO kb.TransformationOperation
(column_id, source_database, expression)
VALUES (@target_col, @source,  '$col1');
INSERT INTO kb.TransformationFromMap
(transformation_op_id, column_id, element_name)
VALUES (SCOPE_IDENTITY(), @src_col, '$col1');

### Load from stage to information tables

In [ ]:
USE KnowledgeBaseDB;

INSERT INTO kb.JoinTable
(join_type, table_1, table_2, condition)
SELECT DISTINCT join_type, table_1, table_2, table_condition
FROM stage.StageJoin

UPDATE sj
SET sj.dwh_join_table_id = jt.join_table_id
FROM stage.StageJoin sj
JOIN kb.JoinTable jt
    ON sj.table_1 = jt.table_1 AND sj.table_2 = jt.table_2

INSERT INTO kb.JoinMap
(join_table_id, element_name, condition)
SELECT DISTINCT sj.dwh_join_table_id, sj.map_element_name, sj.map_condition
FROM stage.StageJoin sj

UPDATE sj
SET sj.dwh_join_map_id = jm.join_map_id
FROM stage.StageJoin sj
JOIN kb.JoinMap jm
    ON jm.join_table_id = sj.dwh_join_table_id AND jm.element_name = sj.map_element_name AND jm.condition = sj.map_condition;

INSERT INTO kb.JoinColumn
(join_map_id, column_id, element_name)
SELECT dwh_join_map_id, column_1_id, column_1_element_name
FROM stage.StageJoin
UNION ALL
SELECT dwh_join_map_id, column_2_id, column_2_element_name
FROM stage.StageJoin;

## Add fill priority of TemplateDB

In [ ]:
USE [KnowledgeBaseDB];

INSERT INTO [kb].[FillOrder]
(table_name, table_schema, order_priority)
VALUES ('University', 'dbo', 1), 
('Course', 'dbo', 11),
('Exam', 'dbo', 12),
('Student', 'dbo', 13),
('Study', 'dbo', 14),
('Registration', 'dbo', 100)

## Show data in information tables

In [ ]:
USE KnowledgeBaseDB;

DECLARE @tname1 VARCHAR(100) = 'Semester';
DECLARE @tname2 VARCHAR(100) = 'Studierende';

DECLARE @db_id INT = (SELECT database_id FROM kb.DatabaseInformation WHERE db_name = 'MedUniWien-Student-System');
DECLARE @table_1 INT = (SELECT table_id FROM kb.TableInformation WHERE database_id = @db_id AND table_name = @tname1)
DECLARE @table_2 INT = (SELECT table_id FROM kb.TableInformation WHERE database_id = @db_id AND table_name = @tname2)

SELECT *
FROM kb.JoinMap jm
JOIN kb.JoinTable jt ON jm.join_table_id = jt.join_table_id
WHERE (table_1 = @table_1 AND table_2 = @table_2) OR (table_1 = @table_2 AND table_2 = @table_1)


In [ ]:
USE KnowledgeBaseDB;

SELECT COUNT(*) as 'Transform Count' FROM kb.TransformationOperation

In [ ]:
USE KnowledgeBaseDB;

SELECT * FROM kb.ColumnCondition cc
JOIN kb.ColumnInformation ci ON ci.column_id = cc.column_id
JOIN kb.TableInformation ti ON ti.table_id = ci.table_id
JOIN kb.DatabaseInformation di ON ti.database_id = di.database_id

In [ ]:
USE KnowledgeBaseDB;

SELECT jm.join_table_id, jt.table_1, jt.table_2, jt.join_type, jm.condition, jc.element_name,ci.table_id, ci.column_id, ci.column_name
FROM kb.JoinMap jm
JOIN kb.JoinColumn jc ON jm.join_map_id = jc.join_map_id
JOIN kb.JoinTable jt ON jt.join_table_id = jm.join_table_id
JOIN kb.ColumnInformation ci ON jc.column_id = ci.column_id
ORDER BY jm.join_table_id;

In [ ]:
USE KnowledgeBaseDB;

SELECT *
FROM kb.TransformationOperation tro
left JOIN kb.TransformationFromMap tfm ON tro.transformation_op_id = tfm.transformation_op_id

In [ ]:
USE KnowledgeBaseDB;

SELECT *
FROM kb.TableSelect ts
JOIN kb.TableSelectElement tse ON ts.table_select_id = tse.table_select_id
LEFT JOIN kb.TableSelectMap tsm ON tse.table_select_element_id = tsm.table_select_element_id